In [2]:
import pandas as pd
import numpy as np
from talib import BBANDS, SAR, RSI, STOCH, EMA, WILLR
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBClassifier

train_df = pd.read_csv("training.csv", names=("open", "high", "low", "close"))
test_df = pd.read_csv("testing.csv", names=("open", "high", "low", "close"))

In [3]:
len(test_df)

19

In [4]:
# Do MinMax normalization
maxValue = train_df.to_numpy().max()
minValue = train_df.to_numpy().min()
diff = maxValue - minValue
train = train_df.transform(lambda x: (x - minValue) / diff)
test = test_df.transform(lambda x: (x - minValue) / diff)

In [5]:
train["upperband"], train["middleband"], train["lowerband"] = BBANDS(train.close.to_numpy())
train["sar"] = SAR(train.high.to_numpy(), train.low.to_numpy())
train["rsi"] = RSI(train.close.to_numpy(), timeperiod=5)
train["slowk"], train["slowd"] = STOCH(train.high.to_numpy(), train.low.to_numpy(), train.close.to_numpy())
train["ema"] = EMA(train.close.to_numpy(), timeperiod=5)
train["willr"] = WILLR(train.high.to_numpy(), train.low.to_numpy(), train.close.to_numpy(), timeperiod=9)
train.isnull().sum()

open          0
high          0
low           0
close         0
upperband     4
middleband    4
lowerband     4
sar           1
rsi           5
slowk         8
slowd         8
ema           4
willr         8
dtype: int64

In [6]:
train_data = train.dropna()
train_data = train_data.reset_index(drop=True)
train_data

,open,high,low,close,upperband,middleband,lowerband,sar,rsi,slowk,slowd,ema,willr
0,0.632121,0.633434,0.610606,0.628889,0.669042,0.647333,0.625624,0.693245,10.760278,35.685147,30.969656,0.648791,-84.066901
1,0.641010,0.657576,0.630505,0.637374,0.666021,0.644121,0.622222,0.683329,24.037590,35.196490,34.271958,0.644985,-70.489978
2,0.635657,0.653535,0.632323,0.648182,0.665042,0.643636,0.622231,0.674602,38.586495,46.049189,38.976942,0.646051,-51.562500
3,0.655455,0.661212,0.640909,0.642626,0.653047,0.640000,0.626953,0.666922,34.358325,57.668859,46.304846,0.644909,-55.539972
4,0.695657,0.727980,0.685354,0.723434,0.724624,0.656101,0.587578,0.610606,78.063128,75.844229,59.854092,0.671084,-3.872633
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1475,0.354040,0.356566,0.347778,0.352929,0.367668,0.344525,0.321382,0.302071,67.065605,87.132992,83.501390,0.345742,-9.775641
1476,0.355152,0.359495,0.350707,0.354343,0.361821,0.350606,0.339391,0.304352,68.454615,83.240226,84.920261,0.348609,-8.121019
1477,0.358081,0.363939,0.351515,0.359293,0.359533,0.354566,0.349598,0.307660,73.368544,78.956152,83.109790,0.352170,-6.845238
1478,0.362727,0.370808,0.356263,0.370202,0.371149,0.357798,0.344447,0.312163,81.365806,83.695902,81.964093,0.358181,-0.981997


In [7]:
y = list()
for i in range(len(train_data)):
    isBull = (train_data["open"][i] > train_data["sar"][i], 
              train_data["open"][i] >= train_data["middleband"][i],
              train_data["rsi"][i] > 50,
              train_data["slowk"][i] >= train_data["slowd"][i],
              train_data["open"][i] >= train_data["ema"][i],
              train_data["willr"][i] > -50)
    if np.count_nonzero(isBull) > 4:
        y.append(2)
    elif np.count_nonzero(isBull) < 2:
        y.append(0)
    else:
        y.append(1)
y = np.array(y, dtype=np.int)

In [8]:
X = list()
for i in range(20, len(train_data)):
    X.append(train_data.loc[i-20:i-1, :].values)
X = np.array(X)

In [9]:
y = y[39:]
len(y)

1441

In [10]:
test = X[-len(test_df):]
len(test)

19

In [12]:
new_X = X[:-len(test_df)]

new_X = new_X.reshape((len(y), -1))

In [13]:
new_X.shape

(1441, 260)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(new_X, y, test_size=0.2, shuffle=False)

In [ ]:
xgb = XGBClassifier(learning_rate=0.1, 
                    objective='multi:softmax',
                    num_class=3,
                    n_estimators=1000, max_depth=1, min_child_weight=2, use_label_encoder=False)
# model = xgb.fit(X_train, y_train,
#                eval_set=[(X_val, y_val)],
#               eval_metric="auc",
#                verbose=True)

In [ ]:
parameters = {
    'max_depth': list(range(1, 10)),
    'min_child_weight': list(range(1, 10)),
    "n_estimators": list(range(100, 1001, 100))
}
gsearch = GridSearchCV(xgb, param_grid=parameters, scoring="f1", cv=2)
gsearch.fit(X_train, y_train,  eval_set=[(X_val, y_val)], eval_metric="auc", verbose=True)
best_parameters = gsearch.best_estimator_.get_params()

In [ ]:
print(best_parameters)

In [ ]:
xgb = XGBClassifier(learning_rate=0.1, n_estimators=30,
                    objective='multi:softmax',
                    num_class=3,
                    max_depth=3, min_child_weight=10, use_label_encoder=False)
model = xgb.fit(X_train, y_train,
                eval_set=[(X_val, y_val)],
                eval_metric="mlogloss",
                verbose=True)

In [ ]:
model.predict(test.reshape(20, -1))

In [ ]:
test["upperband"], test["middleband"], test["lowerband"] = BBANDS(test.close.to_numpy())
# train["ma5"] = MA(train.close.to_numpy(), timeperiod=5)
test["sar"] = SAR(test.high.to_numpy(), test.low.to_numpy())
test["rsi"] = RSI(test.close.to_numpy(), timeperiod=5)
test["slowk"], test["slowd"] = STOCH(test.high.to_numpy(), test.low.to_numpy(), test.close.to_numpy())

In [ ]:
predictions = model.predict(test.values)
len(predictions)

In [ ]:
ans = []
val = 0
for i in range(1, len(predictions)):
    _sum = sum(predictions[i-1:i+1])
    if _sum == 2:
        val = 1 if (val == 1) else (val + 1) 
    elif _sum == 1:
        val = val
    else:
        val = -1 if (val == -1) else val - 1
    ans.append(val)
print(ans, len(ans), sep='\n\n')
with open("./output.csv", "w") as fp:
    for i in range(len(ans)):
        print(ans[i], file=fp)